In [2]:
!pip install transformers faiss-cpu datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible

In [27]:
import os
import faiss
import numpy as np
from flask import Flask, request, jsonify
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from groq import Groq


In [28]:
# --- Load Dataset ---
print("Loading Indian Law dataset...")
dataset = load_dataset("kshitij230/Indian-Law")

# --- Prepare Text Corpus ---
texts = [f"{item['Instruction']} {item['Response']}" for item in dataset['train']]

# --- TF-IDF Vectorization ---
print("Building vector index...")
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)
tfidf_array = tfidf_matrix.toarray()

Loading Indian Law dataset...
Building vector index...


In [29]:
# --- FAISS Index ---
dimension = tfidf_array.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(tfidf_array)

In [31]:
client = Groq(api_key='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')

In [41]:
def retrieve_documents(query, top_k=5):
    query_vec = vectorizer.transform([query]).toarray()
    distances, indices = index.search(query_vec, top_k)
    return [texts[i] for i in indices[0]]

# --- Query Classifier (inner/outer) ---
def classify_query(query):
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {
                    "role": "system",
                    "content": "Classify this query. Return 'inner' if it's a legal question under Indian law or law case based question where legal actions is neede, else return 'outer'."
                },
                {
                    "role": "user",
                    "content": query
                }
            ],
            max_tokens=5
        )
        return response.choices[0].message.content.strip().lower()
    except Exception as e:
        print("Classification error:", e)
        return "outer"

# --- Greeting Bot (outer) ---
def greeting_bot(query):
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "Respond like a polite Indian legal assistant. Handle greetings, small talk, or user emotions."},
                {"role": "user", "content": query}
            ],
            max_tokens=100
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

# --- RAG with Groq (inner) ---
def generate_legal_response(query, context):
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {
                    "role": "system",
                    "content": "You are a legal assistant specializing in Indian law. Use context to generate a helpful answer with relevant laws or articles. Don't go beyond Indian law. give answer in points and in structures way"
                },
                {
                    "role": "user",
                    "content": f"Query: {query}\n\nContext:\n{context}"
                }
            ],
            max_tokens=500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

# --- Final RAG Pipeline ---
def complete_query(query):
    classification = classify_query(query)
    print("Classification:", classification)

    if classification == 'outer':
        return greeting_bot(query)

    # Otherwise, legal (inner)
    context = "\n".join(retrieve_documents(query))
    return generate_legal_response(query, context)

def rag_with_groq(query):
    context = "\n".join(retrieve_documents(query))
    return generate_legal_response(query, context)


In [45]:
def colab_chat():
    print("👨‍⚖️ Wakeel Sahab (Legal RAG Bot) is ready! Type 'q' to quit.\n")
    while True:
        query = input("Ok tell me what happened: ").strip()

        if query.lower() == 'q':
            print("👋 Exiting... Thank you!")
            break

        if ".god" in query.lower():
            print("🔍 Key '.god' detected — Forcing full RAG pipeline.")
            context = "\n".join(retrieve_documents(query))
            r= retrieve_documents(query)
            print("rag fetched data is :",r)
            response = generate_legal_response(query, context)
            print("Mr. Wakeel Sahab:\n", response)
            print("-" * 50)
            continue

        classification = classify_query(query).strip().lower()
        print("Classification:", classification)

        if classification == 'outer':
            response = greeting_bot(query)
        else:
            context = "\n".join(retrieve_documents(query))
            response = response = rag_with_groq(query)


        print("Mr. Wakeel Sahab:\n", response)
        print("-" * 50)

# Call this to start the chat in Colab
colab_chat()


👨‍⚖️ Wakeel Sahab (Legal RAG Bot) is ready! Type 'q' to quit.

Ok tell me what happened: hi
Classification: outer
Mr. Wakeel Sahab:
 Namaste. Hello. It's nice to meet you. How are you doing today? Is there something I can help you with, or would you like to chat for a bit?
--------------------------------------------------
Ok tell me what happened: my wife cheated on me by sleeping with another man what can i do such that i wont have to give allimony in my divorce
Classification: inner
Mr. Wakeel Sahab:
 To address your query regarding divorce and alimony under Indian law, here are the relevant points:

**Regarding Adultery and Divorce:**

1. **Section 497 of the Indian Penal Code (IPC)**: This section defines adultery as an offense where someone has sexual intercourse with a person they know or have reason to believe is the wife of another man, without the consent or connivance of that man. However, the wife is not punishable as an abettor in such cases.
2. **Grounds for Divorce**: Un

In [23]:
print("""
                      ___.   .__                         __
  ___________    _____\_ |__ |  |______________    _____/  |______
 /  ___/\__  \  /     \| __ \|  |  \_  __ \__  \  /    \   __\__  \
 \___ \  / __ \|  Y Y  \ \_\ \   Y  \  | \// __ \|   |  \  |  / __ \_
/____  >(____  /__|_|  /___  /___|  /__|  (____  /___|  /__| (____  /
     \/      \/      \/    \/     \/           \/     \/          \/ """)




                      ___.   .__                         __
  ___________    _____\_ |__ |  |______________    _____/  |______
 /  ___/\__  \  /     \| __ \|  |  \_  __ \__  \  /    \   __\__   \___ \  / __ \|  Y Y  \ \_\ \   Y  \  | \// __ \|   |  \  |  / __ \_
/____  >(____  /__|_|  /___  /___|  /__|  (____  /___|  /__| (____  /
     \/      \/      \/    \/     \/           \/     \/          \/ 


In [46]:

# my wife cheated on me by sleeping with another man what can i do such that i wont have to give allimony in my divorce

